In [ ]:
# Spread:
from quid3 import QConnection, fetch
from datetime import timedelta
from datetime import datetime as dt
from datetime import time
from datetime import date
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dateutil.relativedelta import relativedelta
from xbbg import blp
from eqspydata.vendor import grdb
import time
pd.set_option("display.max_columns", None)



historic_interval = 6       #MINS    time bin for historical data - for 20 days avg
intraday_interval = 3       #MINS    time bin for intraday data
N_days_back = 33            #No. of days back we want to retrieve data from. This includes holidays too.
No_day_avg = 20


otas_grdb_exception = {
    " GY": " GR",
    " SQ": " SM",
    " SE": " SW"
}

def get_otas_grdb_tickers(otas_tickers):
    return [t.replace(key, value) for key, value in otas_grdb_exception.items() for t in otas_tickers]
def get_security_data(tickers=None, rics=None, **kwargs):
    if tickers:
        symbols = get_otas_grdb_tickers(tickers)
        security_data = grdb.get_security_master(symbols=symbols, **kwargs)
    elif rics:
        security_data = grdb.get_security_master(symbols=rics, symbol_type="ric", **kwargs)
    else:
        raise ValueError()
    return security_data['securities']
def get_security_data_df(tickers, **kwargs):
    securities = get_security_data(tickers = tickers, **kwargs)
    # sym = "`" + '`'.join(self.total_member_trades['symbol'].unique().tolist())
    # securities = pd.read_csv('http://grdbqa:8095/secmaster_query/current.csv?query={bloombergId:%20{$in:%20[%22VOD%20LN%22]},isPrimary:true}&include=bloombergId,ric,compositeRic,lnId,compositeLnId,exchangeMic,lotSize,currencyCode,mifid2Lis,tradingHoursCode,exchCountryCode,tickSizeSchema')
    return pd.DataFrame(securities)

def ticker_fx(tickers):
    print("ticker_fx spread called")
    curr = pd.read_html("https://pool.liquidnet.com/api/result/state/6512/export?format=html")[0][["LNCURRENCYCODE", "RATE_TO_USD"]]
    sec_data = get_security_data_df(tickers, fields=["primaryRic", "currencyCode",'marketCapUsd']).drop_duplicates()
    sec_data = sec_data.merge(curr, left_on="currencyCode", right_on="LNCURRENCYCODE", how="left")
    return sec_data

# print(ticker_fx(['VOD LN']))

#HERE INTERVAL IS MEASURED IN MINS
def bars(interval,symlist,start_date,end_date,startTime,endTime):               # N_Days is number of market days before most recent trading day
    with QConnection(app='test', user='tsundar') as conn:
        current = dt.now()
        df = fetch(
            conn,
            'bar',
            dict(
                Type = 'ric',
                symlist=symlist,
                startDate = start_date.date(),
                endDate = end_date.date(),
                barsz = interval,               #interval must be given in an integer number of minutes. Interval is measured in seconds so it must given in multiples of 60
                volumeCol = 'TotalVolume',
                auctionExtension = True,
                startTime = startTime,
                endTime = endTime             #find volume to the current time
            )
        )


    df['time'] = df['time'] + timedelta(hours=1)
    df['date_time'] = df['date']+df['time']
    df = df[['AverageSpreadbps', 'date_time','sym','Value']]
    # df = df.merge(ticker_fx(symList), left_on= ['sym'],right_on=['primaryRic'],how= 'left')

    df['AverageSpreadbps'][df['AverageSpreadbps']<0] = 0
    df = df[df['date_time']<=current]
    return df




def bars_prev(interval, symList,start_date,end_date,startTime,endTime):               # N_Days is number of market days before most recent trading day
    with QConnection(app='test', user='tsundar') as conn:
        current = dt.now()
        df = fetch(
            conn,
            'bar',
            dict(
                Type = 'ric',
                symlist=symList,
                startDate = start_date.date(),
                endDate = end_date.date(),
                barsz = interval,               #interval must be given in an integer number of minutes. Interval is measured in seconds so it must given in multiples of 60
                volumeCol = 'TotalVolume',
                auctionExtension = True,
                startTime = startTime,
                endTime = endTime             #find volume to the current time
            )
        )

    df['time'] = df['time'] + timedelta(hours=1)
    df['date_time'] = df['date']+df['time']
    df = df[['AverageSpreadbps','sym','date_time','time','date','Value']]
    df['AverageSpreadbps'][df['AverageSpreadbps']<0] = 0
    df['AverageSpreadbps'] = df['AverageSpreadbps'].fillna(0)

    return df

def value_weighted_spread(df):
    # single value for avg spread
    total_val = df["Value$"].sum()
    if total_val != 0:
        w_spread =(df['Value$'].multiply(df['AverageSpreadbps'])).sum()/total_val
    else:
        w_spread = 0

    # try:
    #     w_spread =(df['Value$'].multiply(df['AverageSpreadbps'])).sum()/total_val
    # except RuntimeWarning:
    #     w_spread = 0

    return pd.Series({"value_weightedSpread_bps": w_spread})

def mcap_weighted_spread(df):
    total_mcap = df['marketCapUsd'].sum()
    if total_mcap != 0:
        w_spread =(df['marketCapUsd'].multiply(df['AverageSpreadbps'])).sum()/total_mcap
    else:
        w_spread = 0

    return pd.Series({"mcap_weightedSpread_bps": w_spread})



class Spread:

    def __init__(self,names):
        self.names = names
        self.rics = ticker_fx(self.names)
        self.intraday = self.intraday_loader()          #gives the intraday volume - note this does not cumsum. need to call the intraday updater to do this                                                                               #this just gives the 20 day average for particular names passed in      #I'M NOT SURE WE ACTUALLY NEED THIS FUNCTION
        self.historic = self.historic_loader()
        print("initialized spread")

    def update_name_list(self, new_names):
        try:
            self.names = new_names
            self.rics = ticker_fx(new_names)
            self.intraday = self.intraday_loader()
            self.historic = self.historic_loader()
            return True
        except:
            return False

    def intraday_loader(self):

        intra = bars(interval=intraday_interval, symlist=self.rics['primaryRic'].tolist(), start_date=dt.now(),
                     end_date=dt.now(), startTime=dt(2023, 7, 17, 7).time(), endTime=dt.now().time())
        # Now I want a df which combines both the intra dataframe as well as parts of the rics dataframe
        intra = intra.merge(self.rics, left_on='sym', right_on='primaryRic', how='left')
        # intra = intra['Value','date_time','sym','RATE_TO_USD']
        intra['Value$'] = intra['RATE_TO_USD'] * intra['Value'].fillna(method='ffill')
        final_df = intra.groupby('date_time').apply(mcap_weighted_spread)  # take the groupby out of here and put it into the intraday updater function
        final_df['value_weightedSpread_bps'] = intra.groupby('date_time').apply(value_weighted_spread)

        return final_df.reset_index()

    def intraday_updater(self):                 # this function again takes in the names we care about, then make a dataframe of the volume data from the last time of the intra df to the current time. it then concatenates it onto the end of the intraday variable. this way, we dont have to load all of the previous days data.
        if (dt.now().time() > dt(2023, 8, 14, 8).time()):


            last_datetime = max(self.intraday['date_time'])
            next = bars(interval=intraday_interval, symlist=self.rics['primaryRic'].tolist(), start_date=last_datetime,end_date=last_datetime, startTime=((last_datetime) - timedelta(hours=1)).time(),endTime=(dt.now() - timedelta(hours=1)).time())
            if len(next) == 0:
                return self.intraday
                       #accounting for the UTC conversion as in the bars function we added on 1 hour anyway. so here we take off an hour.


            ## take max value of datetime column, then add 2 mins, then concat to self.intraday
            else:
                next = next.merge(self.rics, left_on='sym', right_on='primaryRic', how='left')
                next['Value$'] = next['RATE_TO_USD'] * next['Value'].fillna(method = 'ffill')
                next_final = next.groupby('date_time').apply(mcap_weighted_spread).reset_index()
                next_final['Value_weighted'] = next.groupby('date_time').apply(value_weighted_spread)

                self.intraday = pd.concat([self.intraday,next_final])
                # self.intraday['cum_vol$'] = self.intraday['Value$'].fillna(0).cumsum()
                self.intraday = self.intraday[~self.intraday['date_time'].duplicated(keep = 'first')]           #getting rid of duplicated times contaminating the cumsum
                # self.intraday = self.intraday[self.intraday['date_time'].dt.time<=dt(2023,7,23,17).time()]
                # self.intraday = max(self.intraday['date_time'].time)
                self.intraday['mcap_weightedSpread_bps'] = self.intraday['mcap_weightedSpread_bps'].replace(0,pd.NA).fillna(method = 'ffill')
                self.intraday['value_weightedSpread_bps'] = self.intraday['value_weightedSpread_bps'].replace(0,pd.NA).fillna(method = 'ffill')
                self.intraday= self.intraday[(self.intraday['date_time'].dt.time)<=dt(2023,7,23,16,30).time()]
                return self.intraday

        else:
            last_trade_day = self.historic[self.historic['date'] == self.historic['date'].max()]
            # last_trade_day = last_trade_day.merge(self.rics, left_on='sym', right_on='primaryRic', how='left')
            last_trade_day['Value$'] = last_trade_day['RATE_TO_USD'] * last_trade_day['Value'].fillna(method='ffill')
            final_df = pd.DataFrame()
            final_df['mcap_weightedSpread_bps'] = last_trade_day.groupby('date_time').apply(mcap_weighted_spread)  # take the groupby out of here and put it into the intraday updater function
            final_df['value_weightedSpread_bps'] = last_trade_day.groupby('date_time').apply(value_weighted_spread)
            final_df = final_df.reset_index()
            final_df['time'] = final_df['date_time'].dt.time
            final_df[final_df['time'] <= dt(2023, 7, 23, 16, 30).time()]
            # final_df = final_df[(final_df['date_time'] + dt(2023, 7, 23)).dt.time <= dt(2023, 7, 23, 16, 30).time()]
            self.intraday = final_df


            return self.intraday



    def historic_loader(self):
        # rics = ticker_fx(self.names)
        prev = bars_prev(interval=historic_interval, symList=self.rics['primaryRic'].tolist(),
                         start_date=(dt.now() - timedelta(31)), end_date=(dt.now() - timedelta(days=1)),
                         startTime=dt(2023, 7, 17, 7).time(), endTime=dt(2023, 7, 17, 16).time())
        first_day = prev['date'].unique()[-20]
        prev = prev[prev['date'] >= first_day]
        prev = prev.merge(self.rics, left_on='sym', right_on='primaryRic', how='left')
        return prev


    def historic_updater(self):
        current = dt.now()
        self.historic = self.historic[self.historic['date_time'] >= (current - timedelta(days=No_day_avg))]
        # rics = ticker_fx(self.names)

        if max(self.historic['date_time']).date() != (dt.now() - timedelta(days=1)).date():  # checking whether yesterday's data has been added to the self.historics dataframe

            days_back = 1
            yesterday = bars_prev(interval=historic_interval, symList=self.rics['primaryRic'].tolist(),
                                  start_date=dt.now() - timedelta(days=days_back),
                                  end_date=dt.now() - timedelta(days=days_back),
                                  startTime=dt(2023, 7, 24, 7).time(), endTime=dt(2023, 7, 24, 16).time())

            while len(yesterday) != 0:
                days_back += 1
                yesterday = bars_prev(interval=historic_interval, symList=self.rics['primaryRic'].tolist(),
                                      start_date=dt.now() - timedelta(days=days_back),
                                      end_date=dt.now() - timedelta(days=days_back),
                                      startTime=dt(2023, 7, 24, 7).time(), endTime=dt(2023, 7, 24, 16).time())

            yesterday = yesterday.merge(self.rics, left_on='sym', right_on='primaryRic',how='left')  # joining rics on here. we need to do this here so that the yesterday dataframe is in the same form as the sself.historic dataframe. so that we dont mess it up when we concatenate
            self.historic = pd.concat([self.historic, yesterday])  # here we concat onto the end of self.historic

        self.historic['Value$'] = self.historic['RATE_TO_USD'] * self.historic['Value']  # get dollar value

        return self.historic.sort_values("date_time")


    def historic_average(self):
        t_historic = self.historic_updater()  # historic updater gets called when we click the historic average function
        t_historic = t_historic.groupby(['sym', 'time'], as_index=False, sort=False, dropna=False).mean()       #this line returns the mean values for each unique combination of 'sym' and 'time'
        t_historic_mcap = t_historic.groupby('time', as_index=True, sort=False, dropna=False).apply(mcap_weighted_spread)  #I have made time the index here
        t_historic_value = t_historic.groupby('time', as_index = True).apply(value_weighted_spread)
        final = t_historic_mcap.merge(t_historic_value, left_index=True, right_index=True,how = 'inner')
        # t_historic_final = t_historic_final.sort_values('time')  # not sure why we sort vals by time here tbh   # This is future TJ I think I've figured it out. Its because the time column isn't in chronological order.
        # t_historic_final = t_historic_final.fillna(method='ffill')
        # t_historic_final = t_historic_final[(t_historic_final['time']+dt(2023,7,23)).dt.time<=dt(2023,7,23,16,30).time()]
        final = final.sort_index()
        # final = final.sort_values('time')
        final = final.fillna(method='ffill').reset_index()
        final = final[(final['time']+dt(2023,7,23)).dt.time<=dt(2023,7,23,16,30).time()]

        return final



